# 期末任务

## 一、任务

### 爬取80页前程无忧全国关于大数据的职位内容（4000条）

## 二、获取关于大数据的url

### 1.使用selenium输入大数据并查找

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
chrome_options=Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-gpu')
from selenium.webdriver import ChromeOptions
option = ChromeOptions()
option.add_experimental_option('excludeSwitches',['enable-automation'])
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
url = "https://www.51job.com/"
driver = webdriver.Chrome(executable_path=r"C:\chromedriver_win32\chromedriver.exe", chrome_options=chrome_options, options=option)
driver.get(url)

<ipython-input-2-b98acd0c9428>:2: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path=r"C:\chromedriver_win32\chromedriver.exe", chrome_options=chrome_options, options=option)


In [3]:
driver.find_element_by_xpath('//*[@id="kwdselectid"]').send_keys('大数据') # 寻找搜索框输入搜索的内容“大数据”
driver.find_element_by_xpath('//*[@id="kwdselectid"]').send_keys(Keys.ENTER) # 回车查找

### 2.获得查询后的url

In [4]:
search_url = driver.current_url # 获取查找后的第一页

search_url_list = [] # 一级页面url列表

In [5]:
# 获取80个url（4000条数据）
for i in range(1,81):
    search_url_list.append(search_url.replace('1.html',f'{i}.html'))

## 三、获取职位的信息

### 1.获取json格式内容

In [6]:
import requests

#### 使用xpath获取json格式内容

In [7]:
from lxml import etree
def get_info_xpath(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36'
    }
    response = requests.get(url = url, headers = headers)
    
    # xpath获取json内容
    text = etree.HTML(response.text)
    text = text.xpath('//body/script[@type="text/javascript"][2]/text()')
    info_json_xpath = text[0].split('window.__SEARCH_RESULT__ = ')[-1]

    return info_json_xpath

#### 使用re获取json格式内容

In [8]:
import re
def get_info_re(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36'
    }
    response = requests.get(url = search_url, headers = headers)
    
    # 正则表达式获取json内容
    info_json_re = re.findall('window.__SEARCH_RESULT__ = (.*?)</script>', response.text, re.S)[0]

    return info_json_re

### 2.通过json格式内容获取二级网页url

In [9]:
import json

In [10]:
def get_info_url(info_json):
    info_url_list = [] #二级页面url列表
    
    # 提取二级页面url
    data_json = json.loads(info_json)
    for i in data_json['engine_jds']:
        info_url_list.append(i['job_href'])

    return info_url_list

## 四、获取所需的内容

In [11]:
def get_info(url):
    driver = webdriver.Chrome(executable_path=r"C:\chromedriver_win32\chromedriver.exe",options=option)
    
    driver.get(url)
    driver.refresh() # 刷新页面，反反爬虫
    element = WebDriverWait(driver, 180).until(EC.presence_of_element_located((By.XPATH,'//*[@class="cn"]/h1'))) # 显性等待
    
    job_name = driver.find_element_by_xpath('//*[@class="cn"]/h1').text # 职位名称
    company_name = driver.find_element_by_xpath('//div[@class = "com_msg"]/a/p').text # 公司名称
    basic_info = driver.find_element_by_xpath('//div[@class = "cn"]//p[@class = "msg ltype"]').text # 基本信息（工作经验要求，学历要求，工作地点，招聘人数，发布时间）
    description = driver.find_element_by_xpath('//div[@class = "bmsg job_msg inbox"]').text # 职位描述
    salary = driver.find_element_by_xpath('//div[@class = "cn"]/strong').text # 薪酬
    driver.quit()
    
    basic_info = basic_info.split('|')
    info_list = [" ", " ", " ", " ", " ",] #基本信息列表
    education = ['初中及以下', '高中', '中专', '中技', '大专', '本科', '硕士', '博士', '无学历要求'] # 所有的学历类别
    
    info_list[0] = basic_info[0] # 工作地点
    for info in basic_info[1:]:
        if '经验' in info:
            info_list[1] = info # 工作经验要求
        elif '在校生/应届生' in info :
            info_list[1] = info # 工作经验要求
        elif info in education:
            info_list[2] = info # 学历要求
        elif '人' in info:
            info_list[3] = info #招聘人数
        elif '发布' in info:
            info_list[4] = info #发布时间
            
    sum_info_dict = {
        '职位名称':job_name,
        '公司名称':company_name,
        '工作经验要求':info_list[1],
        '学历要求':info_list[2],
        '工作地点':info_list[0],
        '薪酬':salary,
        '招聘人数':info_list[3],
        '发布时间':info_list[4],
        '职位描述':description
    }

    return sum_info_dict

## 五、保存所需要的数据

In [12]:
import csv

In [13]:
f_csv = open('./大数据招聘数据.csv', encoding='utf-8', newline='', mode='a+')

csv_writer = csv.DictWriter(f_csv, fieldnames=[
    '职位名称',
    '公司名称',
    '工作经验要求',
    '学历要求',
    '工作地点',
    '薪酬',
    '招聘人数',
    '发布时间',
    '职位描述'
])

csv_writer.writeheader()

46

In [14]:
import time

In [17]:
sum=0
for i in range(len(search_url_list)):
    info_json = get_info_re(search_url_list[i]) # 获取一个一级页面的json格式
    info_list = get_info_url(info_json) # 获取一个一级页面所含有的五十个二级页面的url
    
    for url in info_list:
        sum+=1
        
        if sum%40==0: # 每爬取40条数据等待1min
            print('等待 1min 后继续爬虫···')
            time.sleep(60)
        
        info_dict = get_info(url) # 获取所需数据
        csv_writer.writerow(info_dict)
        print('第',sum,'个完成！ url:',url)
        

第 2801 个完成！ url: https://jobs.51job.com/beijing-hdq/136292823.html?s=sou_sou_soulb&t=0_0
第 2802 个完成！ url: https://jobs.51job.com/hangzhou/136290963.html?s=sou_sou_soulb&t=0_0
第 2803 个完成！ url: https://jobs.51job.com/nanjing-yhtq/136288373.html?s=sou_sou_soulb&t=0_0
第 2804 个完成！ url: https://jobs.51job.com/chengdu/132165131.html?s=sou_sou_soulb&t=0_0
第 2805 个完成！ url: https://jobs.51job.com/shenzhen-lhq/136291929.html?s=sou_sou_soulb&t=0_0
第 2806 个完成！ url: https://jobs.51job.com/nanjing-yhtq/136293570.html?s=sou_sou_soulb&t=0_0
第 2807 个完成！ url: https://jobs.51job.com/nanchang/136290623.html?s=sou_sou_soulb&t=0_0
第 2808 个完成！ url: https://jobs.51job.com/shenzhen-ftq/136293307.html?s=sou_sou_soulb&t=0_0
第 2809 个完成！ url: https://jobs.51job.com/changsha/130986779.html?s=sou_sou_soulb&t=0_0
第 2810 个完成！ url: https://jobs.51job.com/zhuhai/131586115.html?s=sou_sou_soulb&t=0_0
第 2811 个完成！ url: https://jobs.51job.com/shanghai-pdxq/134665326.html?s=sou_sou_soulb&t=0_0
第 2812 个完成！ url: https://jobs.51j